In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable

In [2]:
# Data Scource : https://grouplens.org/datasets/movielens/

# we are using encoding because movies name has some special symbols in it.
movies = pd.read_csv('movies.dat',sep='::',header=None,engine='python',encoding='latin-1')
users = pd.read_csv('users.dat',sep='::',header=None,engine='python',encoding='latin-1')
ratings = pd.read_csv('ratings.dat',sep='::',header=None,engine='python',encoding='latin-1')

In [3]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [4]:
movies.shape,users.shape,ratings.shape

((3883, 3), (6040, 5), (1000209, 4))

In [5]:
 # Preparing training and testing set

training_set = pd.read_csv('u1.base',delimiter='\t')
training_set = np.array(training_set, dtype='int')

testing_set = pd.read_csv('u1.test',delimiter='\t')
testing_set = np.array(testing_set, dtype='int')

In [6]:
nb_users = int(max(max(training_set[:,0]),max(testing_set[:,0])))
nb_movies = int(max(max(training_set[:,1]),max(testing_set[:,1])))

In [7]:
nb_users,nb_movies

(943, 1682)

In [8]:
# Converting the data into array with user in the lines and movies in the columns

def convert(data):
  new_data = []
  for id_users in range(nb_users):
    user_list = []
    id_movies = data[:,1][data[:,0]==id_users]
    id_ratings = data[:,2][data[:,0]==id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies-1] = id_ratings
    new_data.append(ratings)
  return new_data


In [9]:
training_set = convert(training_set)
testing_set = convert(testing_set)

In [10]:
# Converting the data into torch tensors

training_set = torch.FloatTensor(training_set)
testing_set = torch.FloatTensor(testing_set)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
# Converting the rating into Binary Rating 1 Like and 0 Dislike

training_set[training_set==0] = -1
training_set[training_set==1] = 0  # Dislike
training_set[training_set==2] = 0  # Dislike
training_set[training_set==3] = 1  # Like
training_set[training_set==4] = 1  # Like
training_set[training_set==5] = 1  # Like

testing_set[testing_set==0] = -1
testing_set[testing_set==1] = 0  # Dislike
testing_set[testing_set==2] = 0  # Dislike
testing_set[testing_set==3] = 1  # Like
testing_set[testing_set==4] = 1  # Like
testing_set[testing_set==5] = 1  # Like

In [12]:
# # Creating the architecture of the Neural Network
# # Restricted Boltzmann Machine is probablistic Graphical Model therefore we are going to build PGM (probablistic Graphical Model).

# class RBM():

#   def __init__(self,n_visible,n_hidden):
#     self.W = torch.randn(n_hidden,n_visible) # mean 0 variance 1
#     # Bias for Hidden nodes, One is given because it expects two dimensions
#     self.a = torch.randn(1,n_hidden)
#     # Bias for visible node, One is given becasue it expects two dimensions
#     self.b = torch.randn(1,n_visible)

#   # we have to approximate log liklihood Gradient for that we need to Apply Gibbs sampling for that need to find probability of hidden node given visible node.
#   def sample_h(self,x):
#     # it will calculate P(h|v) probability of H given V 
#     # P(h|v) is nothing but sigmoid activateion function, applied to weight*x+bias
#     wx = torch.mm(x,self.W.t())
#     # wx+a
#     activation = wx + self.a.expand_as(wx)
#     p_h_given_v = torch.sigmoid(activation)
#     return p_h_given_v, torch.bernoulli(p_h_given_v)

#   # we have to approximate log liklihood Gradient for that we need to Apply Gibbs sampling for that need to find probability of hidden node given visible node.
#   def sample_v(self,y):
#     # it will calculate P(h|v) probability of H given V 
#     # P(h|v) is nothing but sigmoid activateion function, applied to weight*x+bias
#     wy = torch.mm(y,self.W)
#     # wx+a
#     activation = wy + self.b.expand_as(wy)
#     p_v_given_h = torch.sigmoid(activation)
#     return p_v_given_h, torch.bernoulli(p_v_given_h)

#   # this function will take care of Contrastive Divergence
#   def train(self,v0,vk,ph0,phk):
#     #v0 is the input vector containing the ratings of all the movies given by one user.
#     #vk visible nodes after k sampling.
#     #ph0 vector of probabilty 
#     #phk probability of hidden nodes after k iteration.
#     self.W += torch.mm(v0.t(),ph0) - torch.mm(vk.t(),phk)
#     self.b += torch.sum((v0-vk),0)
#     self.a += torch.sum((ph0-phk),0)

class RBM():
  def __init__(self, nv, nh):
    self.W = torch.randn(nh, nv)
    self.a = torch.randn(1, nh)
    self.b = torch.randn(1, nv)
  def sample_h(self, x):
    wx = torch.mm(x, self.W.t())
    activation = wx + self.a.expand_as(wx)
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)
  def sample_v(self, y):
    wy = torch.mm(y, self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)
  def train(self, v0, vk, ph0, phk):
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)



In [13]:
nv = len(training_set[0])
nh = 100
batch_size = 100
model = RBM(nv,nh)

In [14]:
# training the RBM
nb_epochs = 10
for epoch in range(nb_epochs):
  train_loss = 0
  s = 0.
  for id_user in range(0,nb_users-batch_size,batch_size):
    vk = training_set[id_user:id_user+batch_size]
    v0 = training_set[id_user:id_user+batch_size]
    ph0,_ = model.sample_h(v0)
    for k in range(10):
      _,hk = model.sample_h(vk)
      _,vk = model.sample_v(hk)
      vk[v0<0] = v0[v0<0]
    phk,_ = model.sample_h(vk)
    model.train(v0,vk,ph0,phk)
    train_loss += torch.mean(torch.abs(v0[v0>=0]-vk[v0>=0]))
    s += 1
  print('epoch : '+str(epoch)+' loss : '+str(train_loss/s)) 


    

epoch : 0 loss : tensor(0.3451)
epoch : 1 loss : tensor(0.2332)
epoch : 2 loss : tensor(0.2511)
epoch : 3 loss : tensor(0.2475)
epoch : 4 loss : tensor(0.2496)
epoch : 5 loss : tensor(0.2483)
epoch : 6 loss : tensor(0.2461)
epoch : 7 loss : tensor(0.2523)
epoch : 8 loss : tensor(0.2445)
epoch : 9 loss : tensor(0.2494)


In [18]:
# testing the Restriced Boltzmann machine


test_loss = 0
s = 0.
for id_user in range(nb_users):
  v = training_set[id_user:id_user+1]
  vt = testing_set[id_user:id_user+1]
  if len(vt[vt>=0])>0: 
    _,h = model.sample_h(v)
    _,v = model.sample_v(h)
    test_loss += torch.mean(torch.abs(vt[vt>=0]-v[vt>=0]))
    s += 1
print('loss : '+str(test_loss/s)) 


loss : tensor(0.2429)
